# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("cities.csv")
city_data


,Unnamed: 0,City,Country,Lat,Lng,Date,Max Temp,Humidity,Clouds,Wind Speed
0,0,Arraial do Cabo,BR,-22.9661,-42.0278,1646351125,76.96,83,0,9.22
1,1,Port Hardy,CA,50.6996,-127.4199,1646351151,43.75,81,75,11.50
2,2,Morgan City,US,29.6994,-91.2068,1646351151,70.00,45,0,3.44
3,3,Balatan,PH,13.3211,123.2333,1646351152,78.87,79,40,3.38
4,4,Mataura,NZ,-46.1927,168.8643,1646351152,68.49,53,100,2.15
...,...,...,...,...,...,...,...,...,...,...
539,539,Forsytheganj,BY,52.6333,29.7333,1646351430,28.85,90,95,8.81
540,540,Banda Aceh,ID,5.5577,95.3222,1646351430,78.85,77,100,4.70
541,541,Katwijk,NL,52.1942,4.4222,1646351430,41.02,68,0,11.86
542,542,Garissa,KE,-0.4536,39.6401,1646351431,77.67,70,20,9.78


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

In [11]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
narrowed_city_df = city_data.loc[(city_data["Wind Speed"] <= 10) & (city_data["Clouds"] == 0) & \
                                   (city_data["Max Temp"] >= 70) & (city_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Country,Lat,Lng,Date,Max Temp,Humidity,Clouds,Wind Speed
0,0,Arraial do Cabo,BR,-22.9661,-42.0278,1646351125,76.96,83,0,9.22
2,2,Morgan City,US,29.6994,-91.2068,1646351151,70.00,45,0,3.44
43,43,Saint George,US,37.1041,-113.5841,1646350985,78.64,11,0,6.91
81,81,Pasni,PK,25.2631,63.4710,1646351190,71.40,60,0,6.89
239,239,Salalah,OM,17.0151,54.0924,1646351270,73.49,73,0,4.61
302,302,Russell,US,32.3502,-85.1999,1646351127,74.80,22,0,3.44
305,305,Cidreira,BR,-30.1811,-50.2056,1646351305,75.24,85,0,8.97
329,329,Avera,US,33.1940,-82.5271,1646351316,75.88,34,0,6.46
351,351,Quelimane,MZ,-17.8786,36.8883,1646351329,78.75,78,0,0.00
370,370,Júlio de Castilhos,BR,-29.2269,-53.6817,1646351338,70.18,49,0,5.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Arraial do Cabo,BR,-22.9661,-42.0278,
2,Morgan City,US,29.6994,-91.2068,
43,Saint George,US,37.1041,-113.5841,
81,Pasni,PK,25.2631,63.4710,
239,Salalah,OM,17.0151,54.0924,
302,Russell,US,32.3502,-85.1999,
305,Cidreira,BR,-30.1811,-50.2056,
329,Avera,US,33.1940,-82.5271,
351,Quelimane,MZ,-17.8786,36.8883,
370,Júlio de Castilhos,BR,-29.2269,-53.6817,


In [17]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))